In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
from pandas.core.arrays.datetimelike import TimelikeOps

In [2]:
# Setting up accumulator dataframes
master_track_df = pd.DataFrame()
master_album_df = pd.DataFrame()

In [3]:
html = requests.get('https://bandcamp.materiacollective.com/')
data = BeautifulSoup(html.text, 'html.parser')

urls = []

tiles1 = data.findAll('li', {'class': 'featured-item'})
tiles2 = data.findAll('li', {'class': 'music-grid-item square first-four'})
tiles3 = data.findAll('li', {'class': 'music-grid-item square'})

for square in tiles1 + tiles2 + tiles3:
    square = str(square)
    square = square[square.index('href="') + 6:]
    square = square[:square.index('"')]
    if square.index('/album/') == 0:
        square = 'https://bandcamp.materiacollective.com' + square
    urls.append(square)
urls
#urls = [
#    'https://polyarcgames.bandcamp.com/album/moss-original-game-soundtrack',
#    'https://videogamemusicbox.bandcamp.com/album/music-box-classics-hollow-knight',
#    'https://akuparagames.bandcamp.com/album/the-darkside-detective-a-fumble-in-the-dark-original-game-soundtrack',
#    'https://roninopf.bandcamp.com/album/noises-from-shibuya'
#]

['https://polyarcgames.bandcamp.com/album/moss-original-game-soundtrack?label=3683409508&amp;tab=music',
 'https://polyarcgames.bandcamp.com/album/moss-book-ii-original-game-soundtrack?label=3683409508&amp;tab=music',
 'https://valterabreu.bandcamp.com/album/nightslink-original-game-soundtrack?label=3683409508&amp;tab=music',
 'https://ryanike.bandcamp.com/album/shadows-over-loathing-original-game-soundtrack?label=3683409508&amp;tab=music',
 'https://waynestrange.bandcamp.com/album/pursuit-a-great-turnabout-from-the-great-ace-attorney?label=3683409508&amp;tab=music',
 'https://videogamemusicbox.bandcamp.com/album/music-box-classics-kingdom-hearts-ii?label=3683409508&amp;tab=music',
 'https://videogamemusicbox.bandcamp.com/album/music-box-classics-mega-man-2?label=3683409508&amp;tab=music',
 'https://videogamemusicbox.bandcamp.com/album/music-box-classics-final-fantasy-x?label=3683409508&amp;tab=music',
 'https://videogamemusicbox.bandcamp.com/album/music-box-classics-nier-automata?labe

In [4]:
for url in urls:
    html = requests.get(url)
    data = BeautifulSoup(html.text, 'html.parser')

    # Getting album title and artist
    album_title = data.find('h2', {'class': 'trackTitle'})
    artist = str(album_title.next_sibling.next_sibling.find('a'))
    artist = artist[artist.index('>') + 1:-4]
    album_title = str(album_title)[36:-27]

    # Getting prices for each of the formats
    formats = ['Digital Album', 'Compact Disc', 'Record/Vinyl']
    prices = [np.nan] * len(formats)
    for i in range(0,len(formats)):
        price = data.find(text=re.compile('Buy ' + formats[i]))
        # If an element with the right format is found, extract the price and add to array
        # In the case of None, there is no option to buy this way, only either CD or Vinyl
        if price != None:
            prices[i] = str(price.next_element.next_element.find(text=re.compile('\$')))[1:]

    # Getting a list of all of the tags for the album
    tags = []
    for tag_data in data.findAll('a', {'class': 'tag'}):
        tag_data = str(tag_data)
        tags.append(tag_data[tag_data.index('>') + 1:-4])

    # Get the album release date
    from itertools import islice
    release = str(next(islice(data.find('div',{'class': 'tralbumData tralbum-credits'}).children, 0, None)))
    release = pd.to_datetime(release[39:-26])

    # Clean the HTML table, because some include lyrics rows
    tracks_table = data.find('table', {'class': 'track_list track_table'})
    table_clean = '<table class="track_list track_table" id="track_table">\n'
    tracks_table = data.find('table', {'class': 'track_list track_table'})
    for row in tracks_table.findAll('tr', {'class': 'track_row_view linked'}):
        table_clean = table_clean + str(row)
    table_clean = table_clean + '</table>'
    track_df = pd.read_html(table_clean)[0]
    
    # Clean the resulting pandas dataframe
    track_df.rename(columns={1: "Number in Album", 2: "Title"}, inplace=True)
    track_df.drop(columns=[0,3,4], axis=1, inplace=True)
    track_df["Number in Album"] = track_df["Number in Album"].astype(float).astype(int)
    track_df["Title"] = track_df["Title"].apply(lambda x: x.rstrip('abcdefghijklmnopqrstuvwxyz '))
    track_df["Duration"] = track_df["Title"].apply(lambda x: x[-5:])
    track_df["Duration"] = pd.to_timedelta('00:' + track_df["Duration"])
    track_df["Title"] = track_df["Title"].apply(lambda x: x[:-7])
    track_df['Album Title'] = album_title

    # Get cummulative values from this section of the track dataframe
    album_duration = track_df['Duration'].sum()
    number_of_tracks = track_df['Number in Album'].max()

    # Construct the new row for the album dataframe
    album_data = [[album_title, artist, release, number_of_tracks, album_duration, prices[0], prices[1], prices[2], tags]]
    album_cols = ['Album Title', 'Artist', 'Release Date', 'Number of Tracks', 'Album Duration', 'Digital Price', 'CD Price', 'Vinyl Price', 'Tags']
    album_df = pd.DataFrame(album_data, columns=album_cols)

    # Concatenate to accumulators
    master_track_df = pd.concat([master_track_df, track_df])
    master_album_df = pd.concat([master_album_df, album_df])

In [27]:
master_album_df['Digital Price'] = master_album_df['Digital Price'].apply(lambda x: np.nan if x == 'one' else x)
master_track_df.to_csv('track_df.csv',index=False)

In [28]:
master_album_df.to_csv('album_df.csv',index=False)

In [8]:
print('Done :)')

Done :)
